In [24]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [26]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [27]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [28]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,5)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [29]:
num_samples, num_features
tx.shape

(250000, 116)

In [30]:
from compute_gradient import *
from cost import *
from implementations import *

In [31]:
#Step-size

lambdas = np.logspace (-20,1,21)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 116), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [32]:
losses_tr = np.zeros(len(lambdas))
losses_val = np.zeros(len(lambdas))

k_fold = 10
seed = 1
loss_min = np.inf
max_accuracy = 0

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    
    loss_training = 0
    loss_validation = 0
    training_accuracy = 0
    validation_accuracy = 0
    for i,k in enumerate(range(k_fold)):
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
        w, loss_train_k = ridge_regression(y_train, tx_train, lambda_)
        loss_val_k = ridge_loss (y_val,tx_val,w,lambda_)
        loss_training += loss_train_k
        loss_validation += loss_val_k
        training_accuracy += predict_accuracy(y_train,tx_train,w)
        validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
    training_accuracy /= k_fold
    validation_accuracy /= k_fold
    loss_training /= k_fold
    loss_validation /= k_fold
    print(lambda_)
    print(loss_training)
    print(loss_validation)
    print(training_accuracy)
    print(validation_accuracy)
    if (validation_accuracy > max_accuracy):
        weights_star = w
        lambda_star = lambda_
        loss_min = loss_validation
        max_accuracy = validation_accuracy 
    losses_tr[ind_lambda] = loss_training
    losses_val[ind_lambda] = loss_validation



1e-20
0.07696803232428363
93965.9604444564
78.42431111111111
78.396
1.1220184543019653e-19
0.07696805550679874
93965.96044447958
78.42431111111111
78.396
1.2589254117941713e-18
0.07696831561889632
93965.9604447397
78.42431111111111
78.396
1.4125375446227496e-17
0.07697123412463322
93965.9604476582
78.42431111111111
78.396
1.5848931924611109e-16
0.07699059568598468
95273.64716105275
78.43333333333332
78.4076
1.7782794100389227e-15
0.07696729960138818
95102.46286339528
78.42822222222223
78.40360000000001
1.9952623149688828e-14
0.07695361213291636
95108.45802602197
78.42857777777779
78.4048
2.238721138568347e-13
0.07695378517215083
95077.00567638937
78.42804444444445
78.41040000000001
2.5118864315095823e-12
0.07695422759118282
94858.29958940009
78.42835555555556
78.41440000000001
2.8183829312644606e-11
0.07695662137444441
94741.00733795963
78.42777777777778
78.4104
3.1622776601683795e-10
0.07698054617119364
94248.0654335045
78.42391111111111
78.4068
3.5481338923357603e-09
0.07720224282194

In [24]:
lambdas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [34]:
loss_min, lambda_star, max_accuracy

(94858.29958940009, 2.5118864315095823e-12, 78.41440000000001)

In [35]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

78.4028

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre5.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)